In [ ]:
import pandas as pd
from download.downloaders import download
from processing.utils import add_ISO


In [5]:
df = pd.read_csv('data/full_data/result.csv')

In [10]:
df.query("Variable == 'Index' and Year == 2020").sort_values(by="Value")

,ISO,Variable,Value,Aggregation,Year,Country,Continent,UNregion,IncomeLevel,Region
235207,NER,Index,26.27,Index,2020.0,Niger,Africa,Western Africa,Low income,Sub-Saharan Africa
236695,NGA,Index,27.58,Index,2020.0,Nigeria,Africa,Western Africa,Lower middle income,Sub-Saharan Africa
247047,PAK,Index,29.48,Index,2020.0,Pakistan,Asia,Southern Asia,Lower middle income,South Asia
174519,KWT,Index,30.42,Index,2020.0,Kuwait,Asia,Western Asia,High income,Middle East & North Africa
53168,BDI,Index,30.82,Index,2020.0,Burundi,Africa,Eastern Africa,Low income,Sub-Saharan Africa
...,...,...,...,...,...,...,...,...,...,...
20619,AUT,Index,73.18,Index,2020.0,Austria,Europe,Western Europe,High income,Europe & Central Asia
110911,FIN,Index,74.38,Index,2020.0,Finland,Europe,Northern Europe,High income,Europe & Central Asia
85728,CZE,Index,74.71,Index,2020.0,Czech Republic,Europe,Eastern Europe,High income,Europe & Central Asia
87216,DNK,Index,75.25,Index,2020.0,Denmark,Europe,Northern Europe,High income,Europe & Central Asia


In [2]:
def process_EQ2():
    df = (
        pd.read_csv('data/indicator/EQ2/raw/EQ2_GHD.M.csv')
          .query("measure == 'DALYs (Disability-Adjusted Life Years)' and sex == 'Both' and metric == 'Rate'")
          .drop(columns=['sex', 'age', 'cause', 'metric', 'upper', 'lower', 'measure', 'rei'])
          .rename(columns={'location': 'Country', 'year': 'Year', 'val': 'Value'})
    )
    
    df = add_ISO(df)
    return df

In [3]:
test = process_EQ2()

In [4]:
test[test[['ISO', 'Year']].duplicated()]

,Country,ISO,Year,Value


In [ ]:
test = process_EQ2()

In [ ]:
test

In [ ]:
test.query('ISO == "FRA" and Variable == "GJ1"')

In [ ]:
test.query('ISO == "FRA" and Variable == "GN1" and Aggregation =="Indicator_normed"')

In [ ]:
test = pd.read_csv('data/indicator/EQ3/preprocessed/EQ3_origin.M.csv')

In [ ]:
def process_EQ3():
    df = pd.read_csv("data/indicator/EQ3/raw/EQ3_WB.csv")

    df = df[[
        'iso3c', 'country_name', 'total_msw_total_msw_generated_tons_year',
        'population_population_number_of_people'
    ]]

    df['Year'] = 2018

    df['Value'] = df['total_msw_total_msw_generated_tons_year'] / df[
        'population_population_number_of_people']

    df = df.drop(columns=[
        'total_msw_total_msw_generated_tons_year',
        'population_population_number_of_people'
    ]).rename(columns={
        'iso3c': 'ISO',
        'country_name': 'Country',
    })

    return df


In [ ]:
test = process_EQ3()

In [ ]:
test.query('ISO == "DEU"')

In [ ]:
test.query('ISO == "FRA"')

In [ ]:
test

In [ ]:
test = pd.read_csv('data/indicator/SL2/preprocessed/SL2_origin.M.csv')

In [ ]:
test.query("ISO == 'DEU'")

In [ ]:
def process_SL2():
    df = (
        pd.read_csv('data/indicator/SL2/raw/SL2_FAOSTAT.csv')[['Year', 'Area', 'Item', 'Value']]
          
    )
    agri_orga = df.query('Item == "Agriculture area under organic agric."').set_index(["Year", 'Area'])[['Value']]
    agri_tot = df.query('Item == "Agricultural land"').set_index(['Year', 'Area'])[['Value']]
    
    df = (agri_orga / agri_tot * 100).reset_index().dropna().rename(columns={'Area': 'Country'})
    
    
    return df

In [ ]:
test = process_SL2()

In [ ]:
test

In [ ]:
test.query('Country == "Germany"')

In [ ]:
df.Item.unique()

In [ ]:
def process_SL1():

    df = pd.read_csv('data/indicator/SL1/raw/SL1_FAO.M.csv')
    df = df.rename(columns={'Area': 'Country'})
    # df['Value'] = abs(df['Value'])
    df[df['Value'] < 0] = 5 - df[df['Value'] < 0]  # mirror
    return df[['Country', 'Year', 'Value']]

In [ ]:
test = process_SL1()

In [ ]:
def process_SL1():

    df = pd.read_csv('data/indicator/SL1/raw/SL1_FAOSTAT.csv')
    df = df.rename(columns={'Area': 'Country'})
    df['Value'] = abs(df['Value'])
    #df[df['Value'] < 0] = 5 - df[df['Value'] < 0]  # mirror
    return df#[['Country', 'Year', 'Value']]

In [ ]:
test = process_SL1()

In [ ]:
test.query('Country == "France" and Element == "Cropland nutrient flow per unit area"').groupby('Year').Value.sum()

In [ ]:
#pd.read_csv('data/indicator/ME2/preprocessed/ME2_origin.M.csv')

In [ ]:
test = process_ME2()

In [ ]:
test.query('Country == "France"')

In [ ]:
test.query('Country == "Germany"')

In [ ]:
import pandas as pd
from processing.utils import add_ISO

def process_GN1():
    df = (
        pd.read_csv('data/indicator/GN1/raw/GN1_WIPO.M.csv')
          .drop(columns=['Origin', 'Office (Code)']).rename(columns={'Office': 'Country'})
          .melt(id_vars=['Country', 'Field of technology'], var_name='Year', value_name='Value')
    )
    
    df = add_ISO(df)
    
    env = df[df['Field of technology'] == '24 - Environmental technology'].drop(columns=['Field of technology']).set_index(['Country', 'ISO', 'Year'])
    total = df.groupby(['Country', 'ISO', 'Year']).sum()
    
    df = (env / total).reset_index()
    df['Value'] = df.groupby(['ISO'])['Value'].transform(lambda x: x.rolling(7, 1).mean())

    return df

In [ ]:
df = process_GN1()
df.query('ISO == "FRA"')

In [ ]:
df = process_GN1()

In [ ]:
df.query('ISO == "DEU"').plot(kind='line', x='Year', y='Value')

In [ ]:
df

In [ ]:
df = pd.read_csv('data/indicator/GE3/preprocessed/GE3.1_CW.csv')

In [ ]:
test = pd.read_csv('data/full_data/result.csv')

In [ ]:
test.query("Year == 2020 and Aggregation == 'Index'")

In [ ]:
import json
with open('data/indicator/GE3/raw/GE3.1_CW.json', 'r') as outfile:
    data = json.load(outfile)


In [ ]:
config_3 = {
            "GGI_code": "GE3.1",
            "params": {
                "gas_ids[]": [
                    327,
                    328,
                    329
                ],
                "sector_ids[]": [
                    1304,
                    1302
                ],
                "source_ids[]": [111]
            }
        }

In [ ]:
data_json = download('CW', config_3, path=None, restart=False)

In [ ]:
data_json

In [ ]:
from processing.api_preprocessors import CW_Preprocessor, preprocess_raw_dict

In [ ]:
df = preprocess_raw_dict(data_json, CW_Preprocessor)

In [ ]:
df

In [ ]:
test = pd.json_normalize(data_json['data'])

In [ ]:
test

In [ ]:
import requests

def get_CW_API_ids(query_param, API_URL='https://www.climatewatchdata.org/'):
    """

    """            
    url     = f'{API_URL}/api/v1/data/historical_emissions/{query_param}'

    response = requests.get(url, params=query_param, headers={"Accept": "application/json"})
    
    
    data_json = response.json()
    df = pd.json_normalize(data_json['data']) 
    
    return df

In [ ]:
test = get_CW_API_ids(query_param="sectors")


In [ ]:
test

In [ ]:
gases